# Stationary Single-Group Pedestrian Flow - Mathematical Formulation

## 1. Introduction

This notebook implements a stationary (steady-state) single-group pedestrian flow model based on the Hughes model with regularization. The model couples a continuity equation for pedestrian density with a Helmholtz equation for path planning.

---

## 2. Strong Form Formulation

### 2.1 Domain and Variables

- **Domain:** $\Omega \subset \mathbb{R}^2$ (the region where pedestrians move)
- **Boundary:** $\partial\Omega = \Gamma_{\text{walls}} \cup \Gamma_{\text{exits}} \cup \Gamma_{\text{entries}}$

**Primary Variables:**
- $\rho(x,y)$ : pedestrian density [ped/m²]
- $\psi(x,y)$ : transformed potential (via Cole-Hopf transformation: $\psi = e^{-\Phi/\delta}$)
- $u(x,y)$ : velocity field [m/s]

### 2.2 Governing Equations

**Continuity Equation (with diffusion):**
$$
\nabla \cdot (-\varepsilon\nabla\rho + \rho u) = 0 \quad \text{in } \Omega
$$

**Helmholtz Equation (for path planning):**
$$
\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi = 0 \quad \text{in } \Omega
$$

**Velocity Field:**
$$
u = f(\rho) \frac{\nabla\psi}{\|\nabla\psi\|}
$$

where:
- $\varepsilon$ = diffusion coefficient (small, for numerical stability)
- $\delta$ = viscosity parameter (regularization parameter)
- $f(\rho)$ = fundamental diagram (walking speed as function of density)

### 2.3 Fundamental Diagram

We use Weidmann's fundamental diagram:
$$
f(\rho) = \begin{cases}
u_0 & \text{if } \rho = 0 \\
u_0\left(1 - e^{-\gamma(1/\rho - 1/\rho_c)}\right) & \text{if } 0 < \rho \leq \rho_c
\end{cases}
$$

**Parameters:**
- $u_0 = 1.36$ m/s (free-flow walking speed)
- $\rho_c = 8$ ped/m² (critical density)
- $\gamma = 1.913$ ped/m² (shape parameter)

### 2.4 Boundary Conditions

**On Walls ($\Gamma_{\text{walls}}$):**
- Density: $(-\varepsilon\nabla\rho + \rho u)\cdot n = 0$ (no flux through walls)
- Potential: $\nabla\psi\cdot n = 0$ (slip condition)

**On Exits ($\Gamma_{\text{exits}}$):**
- Density: $(-\varepsilon\nabla\rho)\cdot n = 0$ (free outflow)
- Potential: $\psi = 1$ (Dirichlet BC, minimal travel time at exit)

**On Entrances ($\Gamma_{\text{entries}}$):**
- Density: $-(-\varepsilon\nabla\rho + \rho u)\cdot n = g$ (prescribed influx)
- Potential: $(u_0\delta\nabla\psi)\cdot n + \psi = 0$ (Robin BC)

where:
- $n$ = outward normal vector on boundary
- $g$ = inflowing flux density [ped/(m·s)]

---

## 3. Weak Formulation Derivation

### 3.1 Weak Form of Helmholtz Equation

#### Step 1: Function Spaces

Define:
- Solution space: $\psi \in V_\psi = \{v \in H^1(\Omega) : v = 1 \text{ on } \Gamma_{\text{exits}}\}$
- Test space: $\varphi \in V_{0,\psi} = \{v \in H^1(\Omega) : v = 0 \text{ on } \Gamma_{\text{exits}}\}$

#### Step 2: Multiply by Test Function and Integrate

Starting from the strong form:
$$
\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi = 0
$$

Multiply by $\varphi \in V_{0,\psi}$ and integrate over $\Omega$:
$$
\int_\Omega \left(\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi\right) \varphi \, d\Omega = 0
$$

#### Step 3: Integration by Parts (Green's First Identity)

Apply Green's first identity to the Laplacian term:
$$
\int_\Omega \Delta\psi \, \varphi \, d\Omega = -\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS
$$

Substituting back:
$$
-\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega - \int_\Omega \frac{1}{\delta^2 f^2(\rho)} \psi\varphi \, d\Omega + \int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS = 0
$$

#### Step 4: Apply Boundary Conditions

Analyze the boundary integral $\int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS$:

**On $\Gamma_{\text{walls}}$:**
- Boundary condition: $\nabla\psi \cdot n = 0$
- Contribution: $\int_{\Gamma_{\text{walls}}} (\nabla\psi \cdot n) \varphi \, dS = 0$

**On $\Gamma_{\text{exits}}$:**
- Test function: $\varphi = 0$ (homogeneous Dirichlet)
- Contribution: $\int_{\Gamma_{\text{exits}}} (\nabla\psi \cdot n) \varphi \, dS = 0$

**On $\Gamma_{\text{entries}}$:**
- Boundary condition: $(u_0\delta\nabla\psi)\cdot n + \psi = 0 \Rightarrow \nabla\psi \cdot n = -\frac{\psi}{u_0\delta}$
- Contribution: $\int_{\Gamma_{\text{entries}}} (\nabla\psi \cdot n) \varphi \, dS = \int_{\Gamma_{\text{entries}}} \left(-\frac{\psi}{u_0\delta}\right) \varphi \, dS$

#### Step 5: Final Weak Form for Helmholtz Equation

**Find $\psi \in V_\psi$ such that for all $\varphi \in V_{0,\psi}$:**
$$
\boxed{
\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho) \psi\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi\varphi \, dS = 0
}
$$

where $\kappa(\rho) = \frac{1}{\delta f(\rho)}$ is the wave number.

---

### 3.2 Weak Form of Continuity Equation

#### Step 1: Function Spaces

Define:
- Solution space: $\rho \in V_\rho = H^1(\Omega)$ (no Dirichlet BCs)
- Test space: $w \in V_\rho = H^1(\Omega)$

#### Step 2: Multiply by Test Function and Integrate

Starting from the strong form:
$$
\nabla \cdot (-\varepsilon\nabla\rho + \rho u) = 0
$$

Multiply by $w \in V_\rho$ and integrate over $\Omega$:
$$
\int_\Omega \nabla \cdot (-\varepsilon\nabla\rho + \rho u) \, w \, d\Omega = 0
$$

#### Step 3: Integration by Parts (Divergence Theorem)

Apply the divergence theorem with $F = -\varepsilon\nabla\rho + \rho u$:
$$
\int_\Omega \nabla \cdot F \, w \, d\Omega = -\int_\Omega F \cdot \nabla w \, d\Omega + \int_{\partial\Omega} (F \cdot n) w \, dS
$$

This gives:
$$
-\int_\Omega (-\varepsilon\nabla\rho + \rho u) \cdot \nabla w \, d\Omega + \int_{\partial\Omega} ((-\varepsilon\nabla\rho + \rho u) \cdot n) w \, dS = 0
$$

Expanding:
$$
\int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega + \int_{\partial\Omega} ((-\varepsilon\nabla\rho + \rho u) \cdot n) w \, dS = 0
$$

#### Step 4: Apply Boundary Conditions

Analyze the boundary integral $\int_{\partial\Omega} ((-\varepsilon\nabla\rho + \rho u) \cdot n) w \, dS$:

**On $\Gamma_{\text{walls}}$:**
- Boundary condition: $(-\varepsilon\nabla\rho + \rho u) \cdot n = 0$
- Contribution: $\int_{\Gamma_{\text{walls}}} ((-\varepsilon\nabla\rho + \rho u) \cdot n) w \, dS = 0$

**On $\Gamma_{\text{exits}}$:**
- Boundary condition: $(-\varepsilon\nabla\rho) \cdot n = 0$
- This means the diffusive flux vanishes, but the convective flux remains:
  $$(-\varepsilon\nabla\rho + \rho u) \cdot n = (\rho u) \cdot n$$
- Contribution: $\int_{\Gamma_{\text{exits}}} (\rho u \cdot n) w \, dS$

**On $\Gamma_{\text{entries}}$:**
- Boundary condition: $-(-\varepsilon\nabla\rho + \rho u) \cdot n = g \Rightarrow (-\varepsilon\nabla\rho + \rho u) \cdot n = -g$
- Contribution: $\int_{\Gamma_{\text{entries}}} (-g) w \, dS$

#### Step 5: Final Weak Form for Continuity Equation

**Find $\rho \in V_\rho$ such that for all $w \in V_\rho$:**
$$
\boxed{
\int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega + \int_{\Gamma_{\text{exits}}} (\rho u \cdot n) w \, dS = \int_{\Gamma_{\text{entries}}} g w \, dS
}
$$

---

## 4. Coupled System Solution Strategy

The system is coupled and nonlinear. We solve it using a fixed-point iteration (Picard iteration):

### Algorithm

1. **Initialize:** Start with an initial guess $\rho^{(0)}$

2. **Iteration Loop:** For $k = 0, 1, 2, \ldots$ until convergence:

   a. **Solve Helmholtz equation** for $\psi^{(k+1)}$ using $\rho^{(k)}$:
   $$
   \int_\Omega \nabla\psi^{(k+1)} \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho^{(k)}) \psi^{(k+1)}\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi^{(k+1)}\varphi \, dS = 0
   $$

   b. **Compute velocity field** $u^{(k+1)}$:
   $$
   u^{(k+1)} = f(\rho^{(k)}) \frac{\nabla\psi^{(k+1)}}{\|\nabla\psi^{(k+1)}\| + \eta}
   $$
   where $\eta > 0$ is a small regularization parameter to avoid division by zero.

   c. **Solve continuity equation** for $\rho^{(k+1)}$ using $u^{(k+1)}$:
   $$
   \int_\Omega \varepsilon\nabla\rho^{(k+1)} \cdot \nabla w \, d\Omega - \int_\Omega (\rho^{(k+1)} u^{(k+1)}) \cdot \nabla w \, d\Omega + \int_{\Gamma_{\text{exits}}} (\rho^{(k+1)} u^{(k+1)} \cdot n) w \, dS = \int_{\Gamma_{\text{entries}}} g w \, dS
   $$

   d. **Check convergence:** If $\|\rho^{(k+1)} - \rho^{(k)}\| < \text{tol}$, stop; otherwise continue.

---

## 5. Summary of Weak Forms for NGSolve Implementation

### Helmholtz Equation (for $\psi$)
**Bilinear form:**
$$
a_\psi(\psi, \varphi) = \int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho) \psi\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi\varphi \, dS
$$

**Linear form:**
$$
l_\psi(\varphi) = 0
$$

**Essential BC:** $\psi = 1$ on $\Gamma_{\text{exits}}$

### Continuity Equation (for $\rho$)
**Bilinear form:**
$$
a_\rho(\rho, w) = \int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega + \int_{\Gamma_{\text{exits}}} (\rho u \cdot n) w \, dS
$$

**Linear form:**
$$
l_\rho(w) = \int_{\Gamma_{\text{entries}}} g w \, dS
$$

**No essential BCs** (all boundary conditions are natural/Neumann-type)

---

## 6. Next Steps

In the following cells, we will:
1. Import NGSolve libraries
2. Define geometry and mesh
3. Set up finite element spaces
4. Implement the weak forms
5. Implement the iterative solver
6. Visualize results
7. Verify conservation and physical correctness

In [11]:
# ========================================
# Module Import
# ========================================

from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np

In [12]:
# ========================================
# Geometry and BC naming
# ========================================

mesh_maxh = 0.05
Hcol = 1.0  # Height [m]
Hwid = 1.0  # Width [m]

# Top rectangle
rect = Rectangle(Hwid, Hcol).Face()
rect.edges.Min(X).name = "left"
rect.edges.Max(X).name = "right"
rect.edges.Max(Y).name = "entry"
rect.edges.Min(Y).name = "exit"

geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")

Mesh: 514 vertices, 946 elements


## Model Parameters and Constants

We now define all physical parameters and numerical constants for the pedestrian flow model.

In [13]:
# ========================================
# Physical Parameters (from Weidmann)
# ========================================
u0 = 1.36          # Free-flow walking speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

# ========================================
# Regularization Parameters
# ========================================
delta = 0.1        # Viscosity parameter [m]
epsilon = 0.075     # Diffusion coefficient [m²]
eta = 1e-8         # Gradient regularization (avoid division by zero)

# ========================================
# Boundary Conditions
# ========================================
g_inflow = 1.0     # Influx at entrance [ped/(m·s)]

# ========================================
# Numerical Parameters
# ========================================
p_order = 3        # Polynomial order of FEM spaces
max_iter = 100     # Maximum number of Picard iterations
tol = 1e-6         # Convergence tolerance for ||rho^(k+1) - rho^(k)||

print("Model Parameters:")
print(f"  u0 (free-flow speed)     = {u0} m/s")
print(f"  rho_c (critical density) = {rho_c} ped/m²")
print(f"  gamma (Weidmann param)   = {gamma_w} ped/m²")
print(f"\nRegularization:")
print(f"  delta (viscosity)        = {delta} m")
print(f"  epsilon (diffusion)      = {epsilon} m²")
print(f"  eta (grad regularization)= {eta}")
print(f"\nBoundary Conditions:")
print(f"  g_inflow (entrance flux) = {g_inflow} ped/(m·s)")
print(f"\nNumerical Settings:")
print(f"  max_iter                 = {max_iter}")
print(f"  tolerance                = {tol}")

Model Parameters:
  u0 (free-flow speed)     = 1.36 m/s
  rho_c (critical density) = 8.0 ped/m²
  gamma (Weidmann param)   = 1.913 ped/m²

Regularization:
  delta (viscosity)        = 0.1 m
  epsilon (diffusion)      = 0.075 m²
  eta (grad regularization)= 1e-08

Boundary Conditions:
  g_inflow (entrance flux) = 1.0 ped/(m·s)

Numerical Settings:
  max_iter                 = 100
  tolerance                = 1e-06


In [14]:
# ============================================================================
# PARAMETER ANALYSIS - Verify numerical stability before running solver
# ============================================================================
from src import analyze_parameters

print("Running parameter analysis...\n")

results = analyze_parameters(
    u0=u0,
    rho_c=rho_c,
    gamma_w=gamma_w,
    delta=delta,
    epsilon=epsilon,
    mesh=mesh,
    mesh_maxh=mesh_maxh,
    p_order=p_order,
    Hwid=Hwid,
    Hcol=Hcol,
    omega=None  # No underrelaxation in standard Picard
)

# Check if parameters are acceptable
if results['stability_continuity'] in ['EXCELLENT', 'GOOD']:
    print("\n" + "="*60)
    print("✓✓ Parameters are GOOD! Ready to proceed with simulation.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠⚠ WARNING: Parameters may cause stability issues!")
    print("    Consider adjusting epsilon, h, or p before continuing.")
    print("="*60)

Running parameter analysis...

PARAMETER ANALYSIS AND NUMERICAL RECOMMENDATIONS

1. PHYSICAL PARAMETERS

Fundamental Diagram:
  u₀ (free-flow speed)      = 1.360 m/s
  ρc (critical density)     = 8.0 ped/m²
  γ  (Weidmann parameter)   = 1.913 ped/m²

Regularization:
  δ (Helmholtz viscosity)   = 0.100 m
  ε (continuity diffusion)  = 0.075 m²

Speed at different densities:
  f(0.5) = 1.322 m/s
  f(1.0) = 1.105 m/s
  f(2.0) = 0.696 m/s
  f(4.0) = 0.289 m/s
  f(6.0) = 0.104 m/s

2. CHARACTERISTIC LENGTH SCALES

Domain:
  Width × Height = 1.00 × 1.00 m
  L (characteristic) = 1.00 m

Diffusion length (continuity):
  lε = √(ε·L/u₀) = 0.2348 m = 23.48 cm
  Physical meaning: density spreads ~23.5 cm

Screening length (Helmholtz):
  lδ = δ·u₀ = 0.1360 m = 13.60 cm
  Physical meaning: path smoothing ~13.6 cm

Helmholtz wavelength λ = 2π·δ·f(ρ):
  λ(ρ=0.5) = 0.8309 m = 83.09 cm
  λ(ρ=1.0) = 0.6943 m = 69.43 cm
  λ(ρ=2.0) = 0.4375 m = 43.75 cm
  λ(ρ=4.0) = 0.1817 m = 18.17 cm
  λ(ρ=6.0) = 0.0655 m

In [15]:
# ========================================
# Weidmann Fundamental Diagram
# ========================================

def weidmann_speed(rho_val):
    """
    Weidmann fundamental diagram: f(rho)
    
    Returns walking speed as a function of density.
    
    f(rho) = u0 * (1 - exp(-gamma * (1/rho - 1/rho_c)))  for 0 < rho <= rho_c
    f(0)   = u0                                           for rho = 0
    
    Parameters:
    -----------
    rho_val : float or CoefficientFunction
        Pedestrian density [ped/m²]
    
    Returns:
    --------
    speed : float or CoefficientFunction
        Walking speed [m/s]
    """
    # Add small regularization to avoid division by zero
    rho_reg = IfPos(rho_val - 1e-10, rho_val, 1e-10)
    
    # Weidmann formula
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    # Ensure speed doesn't exceed u0 (free-flow speed)
    speed = IfPos(u0 - speed, speed, u0)
    
    # Ensure speed is non-negative
    speed = IfPos(speed, speed, 0.0)
    
    return speed


# Test function for pure Python evaluation (for testing)
def weidmann_speed_python(rho_val):
    """Python version for numerical testing"""
    rho_reg = max(rho_val, 1e-10)
    speed = u0 * (1 - np.exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    speed = min(max(speed, 0.0), u0)
    return speed


# Test the function with some sample values
print("Testing Weidmann fundamental diagram:")
print(f"  f(0.1) = {weidmann_speed_python(0.1):.4f} m/s")
print(f"  f(1.0) = {weidmann_speed_python(1.0):.4f} m/s")
print(f"  f(2.0) = {weidmann_speed_python(2.0):.4f} m/s")
print(f"  f(4.0) = {weidmann_speed_python(4.0):.4f} m/s")
print(f"  f(8.0) = {weidmann_speed_python(rho_c):.4f} m/s  (at critical density)")
print(f"  f(0.0) ≈ {weidmann_speed_python(1e-12):.4f} m/s  (near zero)")

Testing Weidmann fundamental diagram:
  f(0.1) = 1.3600 m/s
  f(1.0) = 1.1050 m/s
  f(2.0) = 0.6963 m/s
  f(4.0) = 0.2892 m/s
  f(8.0) = 0.0000 m/s  (at critical density)
  f(0.0) ≈ 1.3600 m/s  (near zero)


In [16]:
# ========================================
# Finite Element Spaces
# ========================================

# H1 space for density (rho) - no Dirichlet BCs
# Order 2 for better accuracy
fes_rho = H1(mesh, order=p_order)

# H1 space for potential (psi) - Dirichlet BC on exit
# We set dirichlet flag on "exit" boundary
fes_psi = H1(mesh, order=p_order, dirichlet="right")

# Vector H1 space for velocity field (u)
# Dimension 2 for 2D problem
fes_u = H1(mesh, order=p_order, dim=2)

print("Finite Element Spaces:")
print(f"  Density (rho):    {fes_rho.ndof} DOFs")
print(f"  Potential (psi):  {fes_psi.ndof} DOFs ({fes_psi.ndof - fes_rho.ndof} fixed by Dirichlet BC)")
print(f"  Velocity (u):     {fes_u.ndof} DOFs")

# ========================================
# Grid Functions (Solution Variables)
# ========================================

# Current solution
gf_rho = GridFunction(fes_rho, name="density")
gf_psi = GridFunction(fes_psi, name="potential")
gf_u = GridFunction(fes_u, name="velocity")

# Previous iteration (for Picard)
gf_rho_old = GridFunction(fes_rho, name="density_old")

# ========================================
# Initialize with reasonable values
# ========================================

# Initialize density with a small positive value
gf_rho.Set(0.1)
gf_rho_old.Set(0.1)

# Initialize potential with linear interpolation from entry (0) to exit (1)
# Exit is at y=0, entry at y=Hcol
gf_psi.Set(y / Hcol)  

# Set Dirichlet BC for psi: psi = 1 at exit (y=0)
gf_psi.Set(1.0, definedon=mesh.Boundaries("exit"))

# Velocity will be computed from psi
gf_u.Set((0, 0))

print("\nGrid Functions initialized:")
print(f"  rho: min = {min(gf_rho.vec):.4f}, max = {max(gf_rho.vec):.4f}")
print(f"  psi: min = {min(gf_psi.vec):.4f}, max = {max(gf_psi.vec):.4f}")

Finite Element Spaces:
  Density (rho):    4378 DOFs
  Potential (psi):  4378 DOFs (0 fixed by Dirichlet BC)
  Velocity (u):     4378 DOFs

Grid Functions initialized:
  rho: min = -0.0000, max = 0.1000
  psi: min = -0.0000, max = 1.0000


In [17]:
# ========================================
# Helmholtz Equation Weak Form (for ψ)
# ========================================
# Weak form:
# ∫_Ω ∇ψ·∇φ dΩ + ∫_Ω κ²(ρ) ψφ dΩ + ∫_Γ_entry (1/(u0*δ)) ψφ dS = 0
# where κ²(ρ) = 1/(δ²f²(ρ))

print("Setting up Helmholtz equation weak form...")

# Trial and test functions
psi = fes_psi.TrialFunction()
phi = fes_psi.TestFunction()

# Coefficient function for κ²(ρ) based on current density
# κ²(ρ) = 1/(δ² * f²(ρ))
f_rho = weidmann_speed(gf_rho)  # Speed as function of current density
kappa_sq = 1.0 / (delta**2 * f_rho**2)

# Bilinear form for Helmholtz equation
a_psi = BilinearForm(fes_psi, symmetric=True)

# Volume terms: ∫_Ω ∇ψ·∇φ dΩ + ∫_Ω κ²(ρ) ψφ dΩ
a_psi += grad(psi) * grad(phi) * dx
a_psi += kappa_sq * psi * phi * dx

# Robin BC on entry: ∫_Γ_entry (1/(u0*δ)) ψφ dS
a_psi += (1.0 / (u0 * delta)) * psi * phi * ds("entry")

# Linear form (RHS) - is zero for Helmholtz
L_psi = LinearForm(fes_psi)
# (no terms, it's zero)

print("  Helmholtz weak form created")
print(f"  - Gradient term: ∫ ∇ψ·∇φ dx")
print(f"  - Helmholtz term: ∫ κ²(ρ) ψφ dx  with κ² = 1/(δ²f²(ρ))")
print(f"  - Robin BC: ∫ (1/(u0*δ)) ψφ ds('entry')")
print(f"  - RHS: 0")

# ========================================
# Continuity Equation Weak Form (for ρ)
# ========================================
# Weak form:
# ∫_Ω ε∇ρ·∇w dΩ - ∫_Ω (ρu)·∇w dΩ + ∫_Γ_exit (ρu·n)w dS = ∫_Γ_entry g w dS

print("\nSetting up Continuity equation weak form...")

# Trial and test functions
rho = fes_rho.TrialFunction()
w = fes_rho.TestFunction()

# Bilinear form for Continuity equation
a_rho = BilinearForm(fes_rho, symmetric=False)  # Not symmetric due to convection

# Diffusion term: ∫_Ω ε∇ρ·∇w dΩ
a_rho += epsilon * grad(rho) * grad(w) * dx

# Convection term: - ∫_Ω (ρu)·∇w dΩ
# Note: gf_u will be updated in the iteration loop
a_rho += -rho * (gf_u * grad(w)) * dx

# Exit boundary term: ∫_Γ_exit (ρu·n)w dS
# Note: n is the outward normal, automatically handled by ds
n = specialcf.normal(2)  # 2D normal vector
a_rho += rho * (gf_u * n) * w * ds("right")

# Linear form (RHS) - inflow at entry
L_rho = LinearForm(fes_rho)
L_rho += g_inflow * w * ds("entry")

print("  Continuity weak form created")
print(f"  - Diffusion term: ∫ ε∇ρ·∇w dx")
print(f"  - Convection term: -∫ (ρu)·∇w dx")
print(f"  - Exit BC: ∫ (ρu·n)w ds('exit')")
print(f"  - RHS: ∫ g w ds('entry') with g = {g_inflow}")

print("\n✓ Weak forms setup complete")

Setting up Helmholtz equation weak form...
  Helmholtz weak form created
  - Gradient term: ∫ ∇ψ·∇φ dx
  - Helmholtz term: ∫ κ²(ρ) ψφ dx  with κ² = 1/(δ²f²(ρ))
  - Robin BC: ∫ (1/(u0*δ)) ψφ ds('entry')
  - RHS: 0

Setting up Continuity equation weak form...
  Continuity weak form created
  - Diffusion term: ∫ ε∇ρ·∇w dx
  - Convection term: -∫ (ρu)·∇w dx
  - Exit BC: ∫ (ρu·n)w ds('exit')
  - RHS: ∫ g w ds('entry') with g = 1.0

✓ Weak forms setup complete


In [18]:
# ========================================
# Velocity Field Computation
# ========================================
# Computes: u = f(ρ) * ∇ψ / (||∇ψ|| + η)
# where f(ρ) is the Weidmann speed and η is a regularization parameter

def compute_velocity():
    """
    Compute velocity field from current potential and density.
    
    Updates gf_u based on:
        u = f(rho) * grad(psi) / (||grad(psi)|| + eta)
    
    This is the coupling between Helmholtz and Continuity equations.
    """
    # Compute gradient of potential
    grad_psi = grad(gf_psi)
    
    # Compute norm of gradient with regularization
    grad_psi_norm = sqrt(grad_psi[0]**2 + grad_psi[1]**2 + eta)
    
    # Compute speed from current density
    speed = weidmann_speed(gf_rho)
    
    # Compute velocity: u = f(rho) * grad(psi) / ||grad(psi)||
    velocity_cf = speed * grad_psi / grad_psi_norm
    
    # Set the velocity grid function
    gf_u.Set(velocity_cf)
    
    return gf_u


print("Velocity field computation function defined:")
print("  u = f(ρ) * ∇ψ / (||∇ψ|| + η)")
print(f"  - f(ρ): Weidmann fundamental diagram")
print(f"  - η = {eta} (gradient regularization)")
print("\n✓ Velocity computation ready")

Velocity field computation function defined:
  u = f(ρ) * ∇ψ / (||∇ψ|| + η)
  - f(ρ): Weidmann fundamental diagram
  - η = 1e-08 (gradient regularization)

✓ Velocity computation ready


In [19]:
# ========================================
# Picard Iteration Solver
# ========================================

def solve_stationary_picard():
    """
    Solve the coupled stationary pedestrian flow problem using Picard iteration.
    
    Algorithm:
    1. Initialize with current gf_rho
    2. Loop:
       a. Solve Helmholtz for psi (using current rho)
       b. Compute velocity u from psi
       c. Solve Continuity for rho (using current u)
       d. Check convergence
    
    Returns:
    --------
    converged : bool
        True if converged within tolerance
    iteration : int
        Number of iterations performed
    residual : float
        Final residual ||rho^(k+1) - rho^(k)||
    """
    
    print("=" * 60)
    print("Starting Picard Iteration")
    print("=" * 60)
    
    # Store residual history
    residuals = []
    
    for iteration in range(max_iter):
        
        # Store old density for convergence check
        gf_rho_old.vec.data = gf_rho.vec
        
        # ====================================
        # Step a: Solve Helmholtz equation
        # ====================================
        
        # Update kappa_sq based on current density
        f_rho = weidmann_speed(gf_rho)
        kappa_sq = 1.0 / (delta**2 * f_rho**2)
        
        # Rebuild bilinear form (since kappa_sq changed)
        a_psi = BilinearForm(fes_psi, symmetric=True)
        a_psi += grad(psi) * grad(phi) * dx
        a_psi += kappa_sq * psi * phi * dx
        a_psi += (1.0 / (u0 * delta)) * psi * phi * ds("entry")
        
        # Assemble and solve
        a_psi.Assemble()
        L_psi.Assemble()
        
        # Apply Dirichlet BC: psi = 1 on exit
        gf_psi.Set(1.0, definedon=mesh.Boundaries("right"))
        
        # Solve linear system
        gf_psi.vec.data += a_psi.mat.Inverse(fes_psi.FreeDofs()) * (L_psi.vec - a_psi.mat * gf_psi.vec)
        
        # ====================================
        # Step b: Compute velocity field
        # ====================================
        compute_velocity()
        
        # ====================================
        # Step c: Solve Continuity equation
        # ====================================
        
        # Rebuild bilinear form (since u changed)
        a_rho = BilinearForm(fes_rho, symmetric=False)
        a_rho += epsilon * grad(rho) * grad(w) * dx
        a_rho += -rho * (gf_u * grad(w)) * dx
        n = specialcf.normal(2)
        a_rho += rho * (gf_u * n) * w * ds("right")

        with TaskManager():
            # Assemble and solve
            a_rho.Assemble()
            L_rho.Assemble()
            
            # Solve linear system (no Dirichlet BCs for rho)
            gf_rho.vec.data = a_rho.mat.Inverse(fes_rho.FreeDofs()) * L_rho.vec
        
        # ====================================
        # Step d: Check convergence
        # ====================================
        
        # Compute residual: ||rho^(k+1) - rho^(k)||
        residual_vec = gf_rho.vec - gf_rho_old.vec
        residual = sqrt(InnerProduct(residual_vec, residual_vec))
        residuals.append(residual)
        
        # Print progress
        if iteration % 1 == 0 or residual < tol:
            rho_min = min(gf_rho.vec)
            rho_max = max(gf_rho.vec)
            print(f"Iter {iteration:3d}: residual = {residual:.6e}, "
                  f"rho ∈ [{rho_min:.4f}, {rho_max:.4f}]")
        
        # Check convergence
        if residual < tol:
            print("=" * 60)
            print(f"✓ Converged in {iteration + 1} iterations!")
            print(f"  Final residual: {residual:.6e}")
            print("=" * 60)
            return True, iteration + 1, residual, residuals
    
    # Max iterations reached without convergence
    print("=" * 60)
    print(f"✗ Did not converge in {max_iter} iterations")
    print(f"  Final residual: {residual:.6e}")
    print("=" * 60)
    return False, max_iter, residual, residuals


# Run the solver
converged, n_iter, final_residual, residuals = solve_stationary_picard()

Starting Picard Iteration


Iter   0: residual = 1.441591e+01, rho ∈ [-0.7697, 2.2306]
Iter   1: residual = 5.174199e+00, rho ∈ [-0.5569, 3.1246]
Iter   2: residual = 2.347040e+00, rho ∈ [-0.2791, 3.3638]
Iter   3: residual = 1.261428e+00, rho ∈ [-0.2589, 3.4791]
Iter   4: residual = 7.604947e-01, rho ∈ [-0.2294, 3.5314]
Iter   5: residual = 5.506224e-01, rho ∈ [-0.2263, 3.5451]
Iter   6: residual = 4.103086e-01, rho ∈ [-0.2200, 3.5592]
Iter   7: residual = 3.310402e-01, rho ∈ [-0.2207, 3.5579]
Iter   8: residual = 2.632991e-01, rho ∈ [-0.2185, 3.5627]
Iter   9: residual = 2.134091e-01, rho ∈ [-0.2197, 3.5608]
Iter  10: residual = 1.713725e-01, rho ∈ [-0.2184, 3.5627]
Iter  11: residual = 1.381852e-01, rho ∈ [-0.2194, 3.5618]
Iter  12: residual = 1.109510e-01, rho ∈ [-0.2185, 3.5624]
Iter  13: residual = 8.912688e-02, rho ∈ [-0.2193, 3.5622]
Iter  14: residual = 7.141869e-02, rho ∈ [-0.2186, 3.5622]
Iter  15: residual = 5.719594e-02, rho ∈ [-0.2192, 3.5623]
Iter  16: residual = 4.572057e-02, rho ∈ [-0.2187, 3.562

In [20]:
# ========================================
# Visualize Density and Velocity
# ========================================

print("=" * 60)
print("Solution Visualization")
print("=" * 60)

# Compute domain area
domain_area = Integrate(1.0, mesh)

# Create order-1 space for min/max evaluation
# (Order-1 DOFs correspond to vertex values, making min/max meaningful)
fes_p1 = H1(mesh, order=1)

# Print solution statistics
print("\nSolution Statistics:")
print(f"  Density (ρ):")
rho_integral = Integrate(gf_rho * dx, mesh)
rho_mean = rho_integral / domain_area

# Interpolate to P1 for min/max
gf_rho_p1 = GridFunction(fes_p1)
gf_rho_p1.Set(gf_rho)

print(f"    integral = {rho_integral:.6f} ped")
print(f"    mean = {rho_mean:.6f} ped/m²")
print(f"    min ≈ {min(gf_rho_p1.vec):.6f} ped/m² (at vertices)")
print(f"    max ≈ {max(gf_rho_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Domain area: {domain_area:.6f} m²")

print("=" * 60)

# Draw the fields
print("\nGenerating visualizations...")

# 1. Density field
Draw(gf_rho, mesh, "density")
print("  ✓ Density field (ρ)")

# 2. Velocity vector field
Draw(gf_u, mesh, "velocity", vectors={"grid_size": 30})
print("  ✓ Velocity vector field (u)")

print("\n✓ Visualization complete!")

Solution Visualization

Solution Statistics:
  Density (ρ):
    integral = 0.692408 ped
    mean = 0.692408 ped/m²
    min ≈ 0.001115 ped/m² (at vertices)
    max ≈ 3.513031 ped/m² (at vertices)

  Domain area: 1.000000 m²

Generating visualizations...


WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

  ✓ Density field (ρ)


WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

  ✓ Velocity vector field (u)

✓ Visualization complete!
